**🔧 Setup Required**: Before running this notebook, please follow the [setup instructions](../../README.md#setup-instructions) to configure your environment and API keys.

# Multimodal Indexing Pipeline with CLIP (Approach 1)

This notebook demonstrates how to build a multimodal indexing pipeline using direct image embeddings with CLIP.

## Approach 1: Image and Text Embeddings (CLIP)

In this approach, we:

1. **Route Documents**: Separate PDFs and images
2. **Process Text**: Extract and embed text from PDFs
3. **Process Images**: Embed images directly using CLIP
4. **Store Everything**: Combine and store all embeddings

### Advantages:
- Fast processing (no LLM calls for images)
- Good for image-to-image similarity
- Works well with visual queries

### Use Cases:
- Visual search
- Image similarity
- Multimodal retrieval

## Setup

In [1]:
import os
from getpass import getpass

notebook_dir = os.path.dirname(os.path.abspath("2_multimodal_indexing_clip.ipynb")) if os.path.exists("2_multimodal_indexing_clip.ipynb") else os.getcwd()

## Download Sample Data

In [2]:
import urllib.request
import ssl

# Create SSL context
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

def download_file(url, filename):
    """Download file with SSL context"""
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    with urllib.request.urlopen(req, context=ssl_context) as response:
        with open(filename, 'wb') as out_file:
            out_file.write(response.read())

# Create directories
os.makedirs("images", exist_ok=True)
os.makedirs("data_for_indexing", exist_ok=True)

# Download images
download_file(
    "https://upload.wikimedia.org/wikipedia/commons/2/26/Pink_Lady_Apple_%284107712628%29.jpg",
    "images/apple.jpg"
)

# Download PDF
download_file(
    "https://arxiv.org/pdf/1706.03762",
    "data_for_indexing/attention_is_all_you_need.pdf"
)

print("Data downloaded successfully!")

Data downloaded successfully!


## Initialize Pipeline Components

We'll create all the components needed for our multimodal pipeline:

**Storage**:
- `InMemoryDocumentStore`: Stores documents and embeddings with cosine similarity

**Routing**:
- `FileTypeRouter`: Routes files based on MIME types (PDF vs JPEG)

**Conversion**:
- `ImageFileToDocument`: Converts images to documents
- `PyPDFToDocument`: Extracts text from PDFs

**Processing**:
- `DocumentSplitter`: Splits PDFs into page-sized chunks
- `ImagePathFixer`: Custom component to fix image file paths

**Embedding**:
- Text embedder for PDF content
- Image embedder for images

**Storage**:
- `DocumentWriter`: Writes processed documents to the store

In [3]:
from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.converters.image import ImageFileToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders.image import SentenceTransformersDocumentImageEmbedder
from haystack.components.joiners import DocumentJoiner
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack.components.routers.file_type_router import FileTypeRouter
from haystack.components.writers.document_writer import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore

# Create document store
doc_store_clip = InMemoryDocumentStore(embedding_similarity_function="cosine")

# Define components
file_type_router = FileTypeRouter(mime_types=["application/pdf", "image/jpeg"])
final_doc_joiner = DocumentJoiner(sort_by_score=False)
image_converter = ImageFileToDocument()
pdf_converter = PyPDFToDocument()
pdf_splitter = DocumentSplitter(split_by="page", split_length=1)
text_doc_embedder = SentenceTransformersDocumentEmbedder(
    model="sentence-transformers/clip-ViT-L-14",
    progress_bar=False
)
# Don't set root_path - we'll use absolute paths instead
image_embedder_clip = SentenceTransformersDocumentImageEmbedder(
    model="sentence-transformers/clip-ViT-L-14",
    progress_bar=False
)
document_writer = DocumentWriter(doc_store_clip)

print("Components initialized!")

/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Components initialized!


## Create Custom ImagePathFixer Component

The `ImagePathFixer` is crucial because `ImageFileToDocument` only stores the filename, not the full path.
This component restores the full absolute path needed for the image embedder.

In [4]:
from haystack import component
from haystack.dataclasses import Document
from typing import List
import os

@component
class ImagePathFixer:
    """
    Fixes the file paths in image documents to be absolute paths.
    ImageFileToDocument only stores the basename, so we need to restore the full path.
    """
    
    @component.output_types(documents=List[Document])
    def run(self, documents: List[Document]) -> dict:
        """Fix the file paths in documents to be absolute."""
        for doc in documents:
            if "file_path" in doc.meta:
                file_path = doc.meta["file_path"]
                # If it's just a filename without directory, assume it's in the images folder
                if os.path.basename(file_path) == file_path:
                    # Get the notebook directory
                    notebook_dir = os.path.dirname(os.path.abspath("2_multimodal_indexing_clip.ipynb")) if os.path.exists("2_multimodal_indexing_clip.ipynb") else os.getcwd()
                    doc.meta["file_path"] = os.path.join(notebook_dir, "images", file_path)
        
        return {"documents": documents}

print("Custom ImagePathFixer component defined!")

Custom ImagePathFixer component defined!


## Build the Indexing Pipeline

Now we'll connect all components into a complete indexing pipeline. The flow is:

**PDF Branch**:
1. File Router → PDF Converter
2. PDF Converter → PDF Splitter
3. PDF Splitter → Text Embedder

**Image Branch**:
1. File Router → Image Converter
2. Image Converter → Path Fixer (fixes file paths)
3. Path Fixer → Image Embedder

**Final Steps**:
1. Both embedders → Document Joiner
2. Document Joiner → Document Writer → Store

In [5]:
# Build the indexing pipeline
indexing_pipe_clip = Pipeline()
indexing_pipe_clip.add_component("file_type_router", file_type_router)
indexing_pipe_clip.add_component("pdf_converter", pdf_converter)
indexing_pipe_clip.add_component("pdf_splitter", pdf_splitter)
indexing_pipe_clip.add_component("image_converter", image_converter)
indexing_pipe_clip.add_component("image_path_fixer", ImagePathFixer())  
indexing_pipe_clip.add_component("text_doc_embedder", text_doc_embedder)
indexing_pipe_clip.add_component("image_doc_embedder", image_embedder_clip)
indexing_pipe_clip.add_component("final_doc_joiner", final_doc_joiner)
indexing_pipe_clip.add_component("document_writer", document_writer)

# Connect components
indexing_pipe_clip.connect("file_type_router.application/pdf", "pdf_converter.sources")
indexing_pipe_clip.connect("pdf_converter.documents", "pdf_splitter.documents")
indexing_pipe_clip.connect("pdf_splitter.documents", "text_doc_embedder.documents")
indexing_pipe_clip.connect("file_type_router.image/jpeg", "image_converter.sources")
indexing_pipe_clip.connect("image_converter.documents", "image_path_fixer.documents")  
indexing_pipe_clip.connect("image_path_fixer.documents", "image_doc_embedder.documents")
indexing_pipe_clip.connect("text_doc_embedder.documents", "final_doc_joiner.documents")
indexing_pipe_clip.connect("image_doc_embedder.documents", "final_doc_joiner.documents")
indexing_pipe_clip.connect("final_doc_joiner.documents", "document_writer.documents")

🚅 Components
  - file_type_router: FileTypeRouter
  - pdf_converter: PyPDFToDocument
  - pdf_splitter: DocumentSplitter
  - image_converter: ImageFileToDocument
  - image_path_fixer: ImagePathFixer
  - text_doc_embedder: SentenceTransformersDocumentEmbedder
  - image_doc_embedder: SentenceTransformersDocumentImageEmbedder
  - final_doc_joiner: DocumentJoiner
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.application/pdf -> pdf_converter.sources (list[Union[str, Path, ByteStream]])
  - file_type_router.image/jpeg -> image_converter.sources (list[Union[str, Path, ByteStream]])
  - pdf_converter.documents -> pdf_splitter.documents (list[Document])
  - pdf_splitter.documents -> text_doc_embedder.documents (list[Document])
  - image_converter.documents -> image_path_fixer.documents (list[Document])
  - image_path_fixer.documents -> image_doc_embedder.documents (List[Document])
  - text_doc_embedder.documents -> final_doc_joiner.documents (list[Document])
  - image_d

## Visualize the Pipeline

In [6]:
# Visualize the pipeline
os.makedirs("images", exist_ok=True)
indexing_pipe_clip.draw(path="images/multimodal_indexing_clip.png")

![](./images/multimodal_indexing_clip.png)

![](./images/multimodal_indexing_clip.png)

## Run the Indexing Pipeline

In [7]:
# Run the indexing pipeline
# Use absolute paths to avoid path resolution issues

indexing_result = indexing_pipe_clip.run(
    data={"file_type_router": {"sources": [
        os.path.join(notebook_dir, "data_for_indexing/attention_is_all_you_need.pdf"),
        os.path.join(notebook_dir, "images/apple.jpg")
    ]}}
)
indexed_documents = doc_store_clip.filter_documents()
print(f"Indexed {len(indexed_documents)} documents")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Indexed 16 documents


## Inspect the Indexed Documents

In [8]:
# Check the first few documents
for i, doc in enumerate(indexed_documents[:3]):
    print(f"\nDocument {i+1}:")
    print(f"Type: {doc.meta.get('file_path', 'Unknown')}")
    print(f"Has embedding: {doc.embedding is not None}")
    if doc.embedding:
        print(f"Embedding dimension: {len(doc.embedding)}")
    print(f"Content preview: {str(doc.content)[:100]}...")


Document 1:
Type: /Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch4/jupyter-notebooks/multimodal/images/apple.jpg
Has embedding: True
Embedding dimension: 768
Content preview: None...

Document 2:
Type: attention_is_all_you_need.pdf
Has embedding: True
Embedding dimension: 768
Content preview: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and...

Document 3:
Type: attention_is_all_you_need.pdf
Has embedding: True
Embedding dimension: 768
Content preview: 1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural...


## Summary

In this notebook, we:

1. **Built a multimodal indexing pipeline** using CLIP embeddings
2. **Processed both PDFs and images** in a single pipeline
3. **Created embeddings** for both text and images in a shared space
4. **Stored everything** in a document store for later retrieval

### Key Advantages of this Approach:
- **Fast**: No LLM calls needed for images
- **Direct visual understanding**: Images are embedded as-is
- **Good for visual queries**: Works well with image-to-image similarity

### Limitations:
- **Less semantic understanding**: May miss conceptual connections
- **Text queries may be less accurate**: Works better with visual queries

### Next Steps:
- Continue to notebook 3 to see Approach 2 (LLM-based content extraction)
- Compare the two approaches for your use case
- Build a RAG pipeline using this indexed data (notebook 4)